**[LeetCode Link](https://leetcode-cn.com/problems/house-robber-iii/solution/san-chong-fang-fa-jie-jue-shu-xing-dong-tai-gui-hu/)**

## 暴力递归 - 最优子结构
使用**爷爷、两个孩子、4 个孙子**来说明问题，首先来定义这个问题的状态，爷爷节点获取到最大的偷取的钱数
* 首先要明确相邻的节点不能偷，也就是爷爷选择偷，儿子就不能偷了，但是孙子可以偷
* 二叉树只有左右两个孩子，一个爷爷最多 2 个儿子，4 个孙子

根据以上条件，我们可以得出单个节点的钱该怎么算:
* 4 个孙子偷的钱 + 爷爷的钱 VS 两个儿子偷的钱 哪个组合钱多，就当做当前节点能偷的最大钱数。这就是动态规划里面的最优子结构

In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        if not root: return 0
        money = root.val
        if root.left:
            money += self.rob(root.left.left) + self.rob(root.left.right)
        if root.right:
            money += self.rob(root.right.left) + self.rob(root.right.right)       
        return max(money, self.rob(root.left) + self.rob(root.right))

## 记忆化 - 解决重复子问题
针对解法一种速度太慢的问题，经过分析其实现，我们发现爷爷在计算自己能偷多少钱的时候，同时计算了 4 个孙子能偷多少钱，也计算了 2 个儿子能偷多少钱。这样在儿子当爷爷时，就会产生重复计算一遍孙子节点。

于是乎我们发现了一个动态规划的关键优化点
* **重复子问题**

由于二叉树不适合拿数组当缓存，我们这次使用哈希表来存储结果，TreeNode 当做 key，能偷的钱当做 value

In [ ]:
class Solution:
    memo = {}
    def rob(self, root: TreeNode) -> int:
        if not root:
            return 0
        if root in self.memo:
            return self.memo[root]
        withoutRoot = root.val \
            + (self.rob(root.left.left)+self.rob(root.left.right) if root.left else 0)\
            + (self.rob(root.right.left)+self.rob(root.right.right) if root.right else 0)
        withRoot = self.rob(root.left) + self.rob(root.right)
        res = max(withoutRoot,withRoot)
        self.memo[root] = res
        return res

## 终极解法
每个节点可选择偷或者不偷两种状态，根据题目意思，相连节点不能一起偷
* 当前节点选择偷时，那么两个孩子节点就不能选择偷了
* 当前节点选择不偷时，两个孩子节点只需要拿最多的钱出来就行(两个孩子节点偷不偷没关系)

我们使用一个大小为 2 的数组来表示 ```[0, 0]``` 0 代表不偷，1 代表偷

**任何一个节点能偷到的最大钱的状态可以定义为**
* 当前节点选择不偷：当前节点能偷到的最大钱数 = 左孩子能偷到的钱 + 右孩子能偷到的钱
* 当前节点选择偷：当前节点能偷到的最大钱数 = 左孩子选择自己不偷时能得到的钱 + 右孩子选择不偷时能得到的钱 + 当前节点的钱数

In [ ]:

class Solution:
    def rob(self, root: TreeNode) -> int:
        if not root:
            return 0
        return self.helper(root)[1]
        
    def helper(self, root):
        if not root:
            return [0, 0]
        left = self.helper(root.left)
        right = self.helper(root.right)
        withoutRoot = left[1] + right[1]
        withRoot = root.val + left[0] + right[0]
        return [withoutRoot, max(withoutRoot, withRoot)]